In [26]:
import pandas as pd
import numpy as np
import folium

# NHAI Take Home data

In [27]:
traffic= pd.read_csv('nhai_take_home_final.csv')
traffic =traffic.rename(columns = {'Unnamed: 0':'Entry_Count'})

traffic.head()

,Entry_Count,Toll,Vehicle,Type,trxtime
0,0,B,X2,Tempo,2022-08-20 10:44:00
1,1,C,X2,Tempo,2022-08-20 11:05:00
2,2,D,X2,Tempo,2022-08-20 12:00:00
3,0,D,X3,Tempo,2022-08-20 15:31:00
4,1,C,X3,Tempo,2022-08-20 16:13:00


# Road Condition and Other Details Data

In [28]:
toll_rate = pd.read_excel("Road Condition and Other Details.xlsx", sheet_name = 'TollRates') 
toll_rate.head()

,Entry,Exit,Car,Tempo,Truck
0,A,B,100,150,200
1,B,A,100,150,200
2,C,D,75,100,150
3,D,C,75,100,150


In [29]:
tolls = pd.read_excel("Road Condition and Other Details.xlsx", sheet_name = 'Toll') 
tolls.head()

,Toll,Chainage,Lat,Long
0,A,71000,25.874773,74.636765
1,B,190000,25.462954,75.447975
2,C,211000,25.631582,75.549084
3,D,270000,25.905717,75.992700


In [30]:
roughness = pd.read_excel("Road Condition and Other Details.xlsx", sheet_name = 'Roughness') 

# Rectifying column names in the dataframe
new_header = roughness.iloc[0] 
roughness = roughness[1:] 
roughness.columns = new_header 

roughness.head()

,NHNumber,SectionCode,StartChainage,EndChainage,Direction,LaneNumber,LwpIri,RwpIri,LaneIRI,Latitude,Longitude,Latitude,Longitude
1,NH148D,GUL-UNR,69267,69300,Increasing,L1,3.14,2.89,3.015,25.878618,74.620137,25.878549,74.620413
2,NH148D,GUL-UNR,69300,69400,Increasing,L1,2.94,2.69,2.815,25.878549,74.620413,25.878331,74.621369
3,NH148D,GUL-UNR,69400,69500,Increasing,L1,1.26,1.48,1.37,25.878331,74.621369,25.878154,74.622351
4,NH148D,GUL-UNR,69500,69600,Increasing,L1,1.27,1.19,1.23,25.878154,74.622351,25.877974,74.623304
5,NH148D,GUL-UNR,69600,69700,Increasing,L1,1.36,1.25,1.305,25.877974,74.623304,25.877742,74.624284


In [31]:
gis = pd.read_excel("Road Condition and Other Details.xlsx", sheet_name = 'GIS_Data')

# Rectifying column names in the dataframe
new_header = gis.iloc[0] 
gis = gis[1:] 
gis.columns = new_header 
gis.head()

,NHNumber,Description,Chainage,Latitude,Longitude,Altitude,SurveyDate,State,District,EntrustedTo,ROName,OldNHNumber
1,148D,Gulabpura - Uniara,69267,25.878618,74.620137,416.73,14.01.2022,Rajasthan,Sawai Madhopur,NHAI,Jaipur,NaN
2,148D,Gulabpura - Uniara,69272,25.878606,74.620178,416.5,14.01.2022,Rajasthan,Sawai Madhopur,NHAI,Jaipur,NaN
3,148D,Gulabpura - Uniara,69277,25.878593,74.620227,416.49,14.01.2022,Rajasthan,Sawai Madhopur,NHAI,Jaipur,NaN
4,148D,Gulabpura - Uniara,69282,25.878583,74.620272,416.45,14.01.2022,Rajasthan,Sawai Madhopur,NHAI,Jaipur,NaN
5,148D,Gulabpura - Uniara,69287,25.878571,74.620321,416.44,14.01.2022,Rajasthan,Sawai Madhopur,NHAI,Jaipur,NaN


# Problem Statement

# problem statement 1. Visualization of highway alignment along with the location and labels of the Toll Plazas.

In [32]:
tolls_markup = folium.Map(
    zoom_start=8.5,
    location=[25.874773, 74.636765])

Toll = tolls.iloc[0]
folium.Marker(
    location=[Toll['Lat'], Toll['Long']],
).add_to(tolls_markup)

# tolls_markup
for _, Toll in tolls.iterrows():
    folium.Marker(
        location=[Toll['Lat'], Toll['Long']],
    ).add_to(tolls_markup)

# tolls_markup
tolls_markup_label = folium.Map(
    zoom_start=8.5,
    location=[25.874773, 74.636765],
    title = "NH-148D"
)


for _, Toll in tolls.iterrows():
    folium.Marker(
        location=[Toll['Lat'], Toll['Long']],
        popup='Lat:'+str(Toll['Lat'])+',  Long:'+ str(Toll['Long']),
        tooltip=Toll['Toll']
    ).add_to(tolls_markup_label)
    
tolls_markup_label

# problem statement 2. There have been reports that people have been able bypass toll-plazas on entry and it is leading to losses of revenue. There are two toll-able segments in the data (AB/BA) and (CD/DC). The toll is to be a deducted at the entry to a segment. Sometimes the toll-plaza lets some people go without deducting their entry or people have been able to find an unofficial bypass allowing them to skip and entry gate.

In [33]:
# created columns:  toll_count(count of toll transaction) and max_toll(maximum value of toll name)
traffic_grp = traffic.groupby(['Vehicle','Type']).agg(toll_count=('Toll','count'),max_toll=('Toll','max')).reset_index()

# Created a columns - concatenate of toll_count & max_toll.
traffic_grp['con'] = traffic_grp['max_toll'] + traffic_grp['toll_count'].astype(str) 

# Created a binary column informing whether any toll was skipepd
traffic_grp['skipped'] = (traffic_grp['toll_count'] == 3) | (traffic_grp['toll_count'] == 1)

# created 'Entry' column giving skipped entry in abnormal trip
conditions = [(traffic_grp['con'] == 'A1'),(traffic_grp['con'] == 'B1'),\
              (traffic_grp['con'] == 'C1'),(traffic_grp['con'] == 'D1'),\
              (traffic_grp['con'] == 'C3'),(traffic_grp['con'] == 'D3')]
values = ['B', 'A', 'D', 'C', 'D', 'A']
traffic_grp['Entry'] = np.select(conditions, values, default=np.nan)

# Extracted toll rates for each route and for vehicle by transforming the toll_rate df 
toll_rate2 = toll_rate[['Entry','Car','Tempo','Truck']]
toll_rate2 = toll_rate2.set_index('Entry').stack().reset_index(name='Val').rename(columns={'level_1':'Type'})



# Joined the new toll rate df with the traffic df to get the rate against each entry
traffic_grp = pd.merge(traffic_grp, toll_rate2, how='left', on=['Type', 'Entry'])

print("After few operations, the final traffic df is as below (showing Journies with Skipped tolls):")
print(traffic_grp[traffic_grp['skipped']==True].head(),"\n")

print("Solution a. The toll which was skipped(toll-leakage) - {} \n".format(traffic_grp[traffic_grp['skipped'] == True]['Entry'].unique().item()))
print("Solution b. Total trip count that possibly bypassed toll is - {} \n ".format(traffic_grp[traffic_grp['skipped'] == True]['skipped'].count()))
print("Solution c. Revenue loss because of skipped tolls was - {} INR \n".format(traffic_grp['Val'].sum(skipna=True)))

After few operations, the final traffic df is as below (showing Journies with Skipped tolls):
    Vehicle   Type  toll_count max_toll con  skipped Entry    Val
53   X10046  Tempo           3        D  D3     True     A  150.0
62   X10054    Car           3        D  D3     True     A  100.0
110  X10098  Tempo           3        D  D3     True     A  150.0
204  X10182  Truck           3        D  D3     True     A  200.0
244  X10218  Truck           3        D  D3     True     A  200.0 

Solution a. The toll which was skipped(toll-leakage) - A 

Solution b. Total trip count that possibly bypassed toll is - 515 
 
Solution c. Revenue loss because of skipped tolls was - 77650.0 INR 



As we saw that Revenue loss was ~77K for 24 hours only between 4 toll plaza, Analysing the similar revenue loss across the whole country can give the location of the tolls that are lossing the most revenue, and we can then investigate the reason behind those and implement stricter policy for fee collection and penalize those who does not follow them properly.

# Problem Statement 3. The speed of a particular vehicle on a particular stretch in a particular direction depends on multiple factors such as the traffic, vehicle type, condition of the road, direction etc.

# calculated Time for each journey and its respective segments

In [34]:
# copying to new dataframe
traffic_routes=traffic.copy()

## adding Information about subsequent Tolls
traffic_routes['Toll2'] = (traffic_routes.sort_values(by=['trxtime'], ascending=True)
                       .groupby(['Vehicle'])['Toll'].shift(-1))

traffic_routes['Toll3'] = (traffic_routes.sort_values(by=['trxtime'], ascending=True)
                       .groupby(['Vehicle'])['Toll'].shift(-2))

traffic_routes['Toll4'] = (traffic_routes.sort_values(by=['trxtime'], ascending=True)
                       .groupby(['Vehicle'])['Toll'].shift(-3)).fillna('NA')

traffic_routes['rank'] = traffic_routes.sort_values(['trxtime'], ascending=[True]) \
             .groupby(['Vehicle']) \
             .cumcount() + 1

## Creating a column containing the routes for each vehicle
vr=traffic_routes[traffic_routes['rank']==1]
vr2=vr.copy()
vr2['route']=np.where(vr['Toll4']=='NA',vr['Toll4']+'_'+vr['Toll']+'-'+vr['Toll2']+'_'+vr['Toll3'],\
                                        vr['Toll']+'_'+vr['Toll2']+'-'+vr['Toll3']+'_'+vr['Toll4'])
vr2=vr2[['Vehicle','route']]
vr2[['route1', 'route2']] = vr2['route'].str.split('-', 1, expand=True)
vr2=vr2[['Vehicle','route1','route2']]

## Creating multiple routes for each vehicle , one row for AB/BA or CD/DC
route_info = vr2[['Vehicle','route1','route2']]
route_info = route_info.set_index('Vehicle').stack().reset_index(name='routes').rename(columns={'level_1':'route_no'})[['Vehicle','routes']]
route_info[['start', 'end']] = route_info['routes'].str.split('_', 1, expand=True)
route_info=route_info[['Vehicle','start','end']]

## Adding Vehicle Type
route_info2 = pd.merge(route_info, traffic_grp[['Vehicle','Type']], how='left', on=['Vehicle'])
route_info3= pd.merge(route_info2, traffic_routes[['Vehicle','Toll','trxtime']], left_on=  ['Vehicle', 'start'],
                   right_on= ['Vehicle', 'Toll'], 
                   how = 'left').drop(columns=['Toll']).rename(columns={'trxtime':'start_time'})

route_info4= pd.merge(route_info3, traffic_routes[['Vehicle','Toll','trxtime']], left_on=  ['Vehicle', 'end'],
                   right_on= ['Vehicle', 'Toll'], 
                   how = 'left').drop(columns=['Toll']).rename(columns={'trxtime':'end_time'})

## Extracting  Time DIfference for each route stretch
route_info5=route_info4.copy()
route_info5['end_time']=pd.to_datetime(route_info4['end_time'])
route_info5['start_time']=pd.to_datetime(route_info5['start_time'])
route_info5['time_diff']=(route_info5['end_time'] - route_info5['start_time'])/pd.Timedelta(hours=1)
route_info5['route']=route_info5['start']+'_'+route_info5['end']
route_info5=route_info5[['Vehicle','Type','start','end','route','start_time','end_time','time_diff']]

print(" Route Information df view after adding distance info :")
route_info5.head()

 Route Information df view after adding distance info :


,Vehicle,Type,start,end,route,start_time,end_time,time_diff
0,X2,Tempo,NA,B,NA_B,NaT,2022-08-20 10:44:00,NaN
1,X2,Tempo,C,D,C_D,2022-08-20 11:05:00,2022-08-20 12:00:00,0.916667
2,X3,Tempo,D,C,D_C,2022-08-20 15:31:00,2022-08-20 16:13:00,0.700000
3,X3,Tempo,B,A,B_A,2022-08-20 16:29:00,2022-08-20 20:43:00,4.233333
4,X4,Car,A,B,A_B,2022-08-20 09:49:00,2022-08-20 11:42:00,1.883333


#  Added Distance & Speed for each segment

In [35]:
# created new column grp in which both toll A and B has 1 and toll C and D have 2 in grp column
tolls['grp'] = np.where(tolls['Toll'].isin(['A','B']), 1,2)

#new df that contains chainage information about toll
max_km = pd.DataFrame(tolls.groupby(['grp'])['Chainage'].max()).reset_index(['grp'])
max_km.rename(columns = {'Chainage':'maximum_v'}, inplace = True)

#new df that contains chainage information about toll
min_km = pd.DataFrame(tolls.groupby(['grp'])['Chainage'].min()).reset_index(['grp'])
min_km.rename(columns = {'Chainage':'minimum_v'}, inplace = True)

#merged new created dataframe max_km and min_km 
chain_km = pd.merge(max_km, min_km ,how = 'inner', on ='grp')

#merged with tolls df to get distance between tolls
distance_info = pd.merge(tolls, chain_km, how='left', on=['grp'])

#distance between toll in km
distance_info['distance_in_kms'] = (distance_info['maximum_v'] - distance_info['minimum_v'])/1000


#merged with old df tolls_s2 to add distance between tolls in df which contain time diff between toll to get speed
route_info6 = pd.merge(route_info5, distance_info,how = 'left', left_on ='start', right_on ='Toll')

#dropping column which are not usefull in further analysis
route_info7 = route_info6.drop(['start_time','end_time', 'Toll', 'Chainage', 'Lat', 'Long', 'maximum_v', 'minimum_v'], axis = 1)

#speed calculated from distance_in_kms and time_diff columns
route_info7['speed_in_kmph'] = (route_info7['distance_in_kms']/route_info7['time_diff']).round(0)

#Check Outlier routes in the journey and removed them for speed calculation
outliers = ['NA_B','A_C','B_D']
route_info8 = route_info7[~route_info7['route'].isin(outliers)]

print(" Route Information df view after adding Distance & Speed info :")
route_info8.head()

 Route Information df view after adding Distance & Speed info :


,Vehicle,Type,start,end,route,time_diff,grp,distance_in_kms,speed_in_kmph
1,X2,Tempo,C,D,C_D,0.916667,2.0,59.0,64.0
2,X3,Tempo,D,C,D_C,0.700000,2.0,59.0,84.0
3,X3,Tempo,B,A,B_A,4.233333,1.0,119.0,28.0
4,X4,Car,A,B,A_B,1.883333,1.0,119.0,63.0
5,X4,Car,C,D,C_D,1.066667,2.0,59.0,55.0


# Solution : Average and Median speed for different segment and different direction

In [36]:
print('Average and Median in diff segment and diff direction\n')
#average speed and median speed
route_info8.groupby(['route']).agg(average_speed=('speed_in_kmph','mean'),median_speed=('speed_in_kmph','median')).reset_index().round(2).head()


Average and Median in diff segment and diff direction



,route,average_speed,median_speed
0,A_B,57.35,55.0
1,B_A,36.15,36.0
2,C_D,64.48,60.0
3,D_C,50.98,51.0


# Solution : Average and Median speed of all type vehicles for different segment and different direction

In [37]:
print('Average speed of different type vehicles in diff segment and diff direction\n')
#average speed of different type of vehicles
route_info8.groupby(['route','Type']).agg(average_speed=('speed_in_kmph','mean')).reset_index().round(2).head(10)

Average speed of different type vehicles in diff segment and diff direction



,route,Type,average_speed
0,A_B,Car,57.37
1,A_B,Tempo,57.26
2,A_B,Truck,57.42
3,B_A,Car,36.08
4,B_A,Tempo,36.24
5,B_A,Truck,36.12
6,C_D,Car,64.40
7,C_D,Tempo,65.01
8,C_D,Truck,64.03
9,D_C,Car,65.03


# Solution : Quartile speed for different segment and different direction

In [38]:
#created functions to calculate all quartiles
def q1(x):
    return x.quantile(0.25)

def q2(x):
    return x.quantile(0.5)

def q3(x):
    return x.quantile(0.75)

def q4(x):
    return x.quantile(1)

#calculated 1st, 2rd, 3rd and 4th quartiles by group
vals = {'speed_in_kmph': [q1, q2, q3, q4]}

print('Quartile(25%, 50%, 75% & 100%) Speed for diff direction and diff segments\n')
#quartile speed for diff segments and diff direction
route_info8.groupby('route').agg(vals).reset_index()

Quartile(25%, 50%, 75% & 100%) Speed for diff direction and diff segments



route speed_in_kmph                    
                   q1    q2    q3      q4
0   A_B          49.0  55.0  63.0   166.0
1   B_A          34.0  36.0  38.0    60.0
2   C_D          51.0  60.0  72.0   885.0
3   D_C          26.0  51.0  64.0  1770.0

# Question 4. Relationship between average speed of all type of vehicle and LaneIRI value in different segments and direction ?

# Solution : Based on the ovbservation that Higher IRI was impacting the speed in BA strectch but not in CD segment and there could be other factors impacting the speed such as traffic. So, Based on the given data it would be impossible to comment on the relation between IRI score and average speed.

In [39]:
# dictionary conatins key(segment and direction) and value(direction and location coordinates)
direct_dict={'A_B':['Increasing',71000,189900],'B_A':['Decreasing',71100,190000],\
             'C_D':['Increasing',211000,269900],'D_C':['Decreasing',211100,270000]}
# blank dict to save average speed
iri_dict={}
for ele,ele_list in direct_dict.items():
    iri_dict[ele]=roughness[(roughness['Direction'] == ele_list[0]) \
                                & (roughness['StartChainage'].between(ele_list[1],ele_list[2]))]['LaneIRI'].mean().round(2)
print('Average speed of all type of vehicle in different segment and different direction\n')
#print average speed 
print(route_info8.groupby(['route']).agg(average_speed=('speed_in_kmph','mean')).round(0))
#print average Lane IRI
print('\nAverage value of LaneIRI in different segments and different direction\n')
print(iri_dict)

Average speed of all type of vehicle in different segment and different direction

       average_speed
route               
A_B             57.0
B_A             36.0
C_D             64.0
D_C             51.0

Average value of LaneIRI in different segments and different direction

{'A_B': 2.18, 'B_A': 5.24, 'C_D': 2.21, 'D_C': 1.89}
